In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from datetime import timedelta
from datetime import datetime
from datetime import timezone
import time
import sqlalchemy

# Import Machine Learning Dependencies
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
import io
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
from io import StringIO

In [3]:
#pip install sodapy

Note: you may need to restart the kernel to use updated packages.


In [4]:
from sodapy import Socrata
from config import MyAppToken
from config import username
from config import password

In [5]:
engine = create_engine("sqlite:///final_project")

In [6]:
#load in the data via API call
client = Socrata('data.austintexas.gov', MyAppToken, username, password)

In [7]:
results = client.get("xwdj-i9he", limit=1160000)

In [8]:
results_df = pd.DataFrame.from_records(results)

In [27]:
results_df.sample(25)

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_zip_code,Created Date,Closed Date,Request Year,Request Month,Time of Day,Open Time,Open Time delta,Total Seconds,Response Time
594547,"9538 NEENAH AVE, AUSTIN, 78717",6,Spot311 Interface,2018-12-12T14:56:06.000,3105537.94419567,Closed,Found Animal Report - Keep,NaN,404Q,-97.76476397,...,78717,2018-04-11 07:27:56,2018-12-12 14:56:06,2018,April,Mid Morning,245 days 07:28:10,245 days 07:28:10,21194890.0,Slow Response
575583,"MONTOPOLIS DR & GROVE DR, AUSTIN, TX 78741",3,Phone,2018-12-06T01:02:35.000,3130751.24745866,Closed,Mowing Medians,TRAVIS,616P,-97.69138277,...,78741,2018-05-23 16:53:34,2018-12-06 01:02:35,2018,May,Evening,196 days 08:09:01,196 days 08:09:01,16963741.0,Slow Response
351603,"1625 BARTON SPRINGS RD, AUSTIN, TX 78704",5,Phone,2019-09-08T00:38:51.000,3108339.88587617,Closed,Injured / Sick Animal,TRAVIS,584Y,-97.76189207,...,78704,2019-09-07 20:10:45,2019-09-08 00:38:51,2019,September,Night,0 days 04:28:06,0 days 04:28:06,16086.0,Quick Response
546852,"500 W 2ND ST, AUSTIN, TX 78701",9,Spot311 Interface,2018-07-30T13:33:51.000,3112302.90449945,Duplicate (closed),School Zone Flasher - Timing/Maintenance,TRAVIS,585W,-97.74923236,...,78701,2018-07-24 08:19:49,2018-07-30 13:33:51,2018,July,Late Morning,6 days 05:14:02,6 days 05:14:02,537242.0,Slow Response
471941,"702 NECHES ST, AUSTIN, TX 78701",9,Phone,2019-07-15T01:01:18.000,3115852.00492256,Closed,Pavement Failure,TRAVIS,585T,-97.73793634,...,78701,2019-01-15 09:45:12,2019-07-15 01:01:18,2019,January,Late Morning,180 days 15:16:06,180 days 15:16:06,15606966.0,Slow Response
327931,"EBERHART LN & CIRCLE S RD, AUSTIN, TX",2,Phone,2019-10-23T12:16:48.000,3104074.75143248,Closed,Debris in Street,TRAVIS,644X,-97.77712761,...,78745,2019-10-23 06:49:51,2019-10-23 12:16:48,2019,October,Mid Morning,0 days 05:26:57,0 days 05:26:57,19617.0,Quick Response
484075,"4607 DOVE SPRINGS DR, AUSTIN, TX 78744",2,Phone,2019-01-17T01:02:18.000,3112615.33495891,Closed,Sign - Traffic Sign Maintenance,TRAVIS,674D,-97.75038615,...,78744,2018-12-12 02:19:59,2019-01-17 01:02:18,2018,December,Early Morning,35 days 22:42:19,35 days 22:42:19,3105739.0,Slow Response
551913,"1116 W 6TH ST, AUSTIN, TX 78703",9,Phone,2018-08-09T18:17:04.000,3110074.5,Closed,Bicycle Issues,TRAVIS,584V,-97.75610937,...,78703,2018-07-12 15:35:10,2018-08-09 18:17:04,2018,July,Afternoon,28 days 02:41:54,28 days 02:41:54,2428914.0,Slow Response
608529,"1801 EXPOSITION BLVD, AUSTIN, TX 78703",10,Phone,2018-03-25T10:26:22.000,3105423.25,Closed,Aquatics Hotline Inquiry,TRAVIS,584C,-97.77032067,...,78703,2018-03-05 08:42:52,2018-03-25 10:26:22,2018,March,Late Morning,20 days 01:43:30,20 days 01:43:30,1734210.0,Slow Response
577080,"2400 WICKERSHAM LN BUILDING 14, AUSTIN, TX",3,Web,2018-05-21T09:56:13.000,3120814.24825471,Closed,Austin Code - Request Code Officer,TRAVIS,615Y,-97.72329993,...,78741,2018-05-21 09:46:50,2018-05-21 09:56:13,2018,May,Late Morning,0 days 00:09:23,0 days 00:09:23,563.0,Quick Response


In [10]:
len(results_df)

1154031

In [9]:
results_df["Created Date"] = pd.to_datetime(results_df["sr_created_date"])
results_df['Closed Date'] = pd.to_datetime(results_df['sr_closed_date'])

In [10]:
results_df['Request Year'] = pd.DatetimeIndex(results_df['Created Date']).year

In [11]:
results_df['Request Month'] = pd.DatetimeIndex(results_df['Created Date']).month

In [12]:
results_df['Time of Day'] = pd.DatetimeIndex(results_df['Created Date']).hour

In [13]:
#filter the df by Request Year
results_df = results_df[(results_df['Request Year'] == 2018) | (results_df['Request Year'] == 2019)]

In [14]:
results_df['Open Time'] = (results_df['Closed Date'] - results_df['Created Date'])

In [15]:
results_df['Open Time delta'] = pd.to_timedelta(results_df['Open Time'])
results_df['Total Seconds'] = (results_df['Open Time delta']).dt.total_seconds()

In [16]:
results_df = results_df[(results_df['Total Seconds'] < 31540000)]

In [17]:
median = results_df['Total Seconds'].median(axis=0)

In [20]:
#median

In [18]:
conditions = [
    (results_df['Total Seconds'] <= median),
    (results_df['Total Seconds'] > median)
    ]
values = ['Quick Response', 'Slow Response']
results_df['Response Time'] = np.select(conditions, values)

In [19]:
conditions = [
    (results_df['Request Month'] == 1),
    (results_df['Request Month'] == 2),
    (results_df['Request Month'] == 3),
    (results_df['Request Month'] == 4),
    (results_df['Request Month'] == 5),
    (results_df['Request Month'] == 6),
    (results_df['Request Month'] == 7),
    (results_df['Request Month'] == 8),
    (results_df['Request Month'] == 9),
    (results_df['Request Month'] == 10),
    (results_df['Request Month'] == 11),
    (results_df['Request Month'] == 12)
]
values = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
results_df['Request Month'] = np.select(conditions, values)

In [26]:
#categorize the Hour variable
conditions = [
    (results_df['Time of Day'] >= 0) & (results_df['Time of Day'] < 4),
    (results_df['Time of Day'] >= 4) & (results_df['Time of Day'] < 8),
    (results_df['Time of Day'] >= 8) & (results_df['Time of Day'] < 12),
    (results_df['Time of Day'] >= 12) & (results_df['Time of Day'] < 16),
    (results_df['Time of Day'] >= 16) & (results_df['Time of Day'] < 20),
    (results_df['Time of Day'] >= 20) & (results_df['Time of Day'] <= 23)
    ]
values = ['Early Morning', 'Mid Morning', 'Late Morning', 'Afternoon', 'Evening', 'Night']
results_df['Time of Day'] = np.select(conditions, values)

TypeError: '>=' not supported between instances of 'str' and 'int'

In [25]:
timecheck = results_df['Time of Day'].value_counts()
timecheck

Late Morning     107091
Afternoon         96948
Evening           69938
Night             25225
Mid Morning       21088
Early Morning      6027
Name: Time of Day, dtype: int64

In [23]:
results_df.head()

,sr_location,sr_location_council_district,sr_method_received_desc,sr_closed_date,sr_location_x,sr_status_desc,sr_type_desc,sr_location_county,sr_location_map_page,sr_location_long,...,sr_location_zip_code,Created Date,Closed Date,Request Year,Request Month,Time of Day,Open Time,Open Time delta,Total Seconds,Response Time
297886,"6510 BERKMAN DR, AUSTIN, TX 78723",1,Phone,2020-01-02T18:13:51.000,3130569.45570825,Closed,Loud Commercial Music,TRAVIS,556K,-97.68991552,...,78723,2019-12-31 23:57:21,2020-01-02 18:13:51,2019,December,23,1 days 18:16:30,1 days 18:16:30,152190.0,Slow Response
297887,"1800 E STASSNEY LN, AUSTIN, TX 78744",2,Spot311 Interface,2020-01-02T18:13:45.000,3109221.74865566,Closed,Loud Commercial Music,TRAVIS,644U,-97.76081753,...,78744,2019-12-31 23:44:51,2020-01-02 18:13:45,2019,December,23,1 days 18:28:54,1 days 18:28:54,152934.0,Slow Response
297888,"400 E 7TH ST, AUSTIN, TX 78701",9,Web,2020-01-02T07:03:59.000,3115538.40203928,Closed,Street Light Issue- Address,TRAVIS,585T,-97.7389242,...,78701,2019-12-31 23:35:12,2020-01-02 07:03:59,2019,December,23,1 days 07:28:47,1 days 07:28:47,113327.0,Slow Response
297889,"200 E 15TH ST, AUSTIN, TX 78701",1,Spot311 Interface,2020-01-02T09:06:02.000,3115838.49761388,Duplicate (closed),ARR Dead Animal Collection,TRAVIS,585P,-97.737741,...,78701,2019-12-31 23:17:46,2020-01-02 09:06:02,2019,December,23,1 days 09:48:16,1 days 09:48:16,121696.0,Slow Response
297890,"200 SAN JACINTO BLVD, AUSTIN, TX 78701",9,Phone,2020-01-02T18:13:44.000,3114645.13713938,Closed,Loud Commercial Music,TRAVIS,585W,-97.74186188,...,78701,2019-12-31 23:03:52,2020-01-02 18:13:44,2019,December,23,1 days 19:09:52,1 days 19:09:52,155392.0,Slow Response


In [24]:
#print(results_df['Total Seconds'].max())

In [25]:
#results_df.loc[results_df['Total Seconds'] == 109367883]

In [26]:
#np.nanpercentile(results_df['Total Seconds'], 99.35) 

In [27]:
#response = results_df['Response Time'].value_counts()
#response

In [30]:
code_types = ['Austin Code - Request Code Officer', 'Code Compliance', 'Code Compliance', 'Construction Ordinance SR', 'Austin Code - Signs/Billboards', 'Austin Code - Short Term Rental Complaint SR', 'Loud Commercial Music', 'Loud Music', 'Austin Code - Coronavirus', 'Austin Code - Short Term Rental (STR) Appointment', 'Austin Code - Construction Rest Break Complaint', 'Austin Code - Construction Ordinance SR', 'Short Term Rental Complaint SR',  'Austin Code - Woodridge Apt']
animal_types = ['Animal - Proper Care', 'Dangerous Animal - Except Dogs', 'Animal Bite',  'Animal Control - Assistance Request', 'Loose Dog', 'Found Animal - Pick Up', 'Loose Animal Not Dog', 'Injured / Sick Animal',  'Found Animal Report - Keep',  'Wildlife Exposure', 'Animal In Vehicle', 'Dangerous/Vicious Dog Investigation', 'Coyote Complaints', 'Animal Trapped in Storm Drain', 'Animal Roadside Sales', 'Loose Animal (not dog)', 'Bat Complaint']
public_types = ['Pothole Repair', 'Obstruction - Urban Forestry', 'Guardrail - New', 'Sidewalk Repair', 'Traffic Signal - Maintenance', 'Traffic Signal - Dig Tess Request',  'Street Light Issue- Address', 'Street Lights New', 'Pavement Failure', 'Sidewalk/Curb Ramp/Route - NEW',  'Alley & Unpaved Street Maintenance',  'Curb/Gutter Repair', 'Bridge Repair', 'Guardrail New/Repair', 'Street Light Issue- Multiple poles/multiple streets', 'Fence/Wall Repair' 'Street Resurfacing', 'Tree Issue ROW/Emergency (PW)', 'Tree Issue ROW/Maintenance (PW)', 'Street Resurfacing Inquiry', 'Guardrail Repair', 'Barricade Request', 'Road Sanding Request']
resource_types = ['ARR Dead Animal Collection', 'ARR Missed Recycling', 'Debris in Street', 'ARR Missed Recycling',  'ARR Missed Yard Trimmings/Compost', 'ARR Brush and Bulk', 'ARR Street Sweeping', 'ARR Missed Yard Trimmings /Organics',  'Ordinance Universal Recycling',  'Ordinance Single-Use Carryout Bags', 'ARR Dumpster', 'Roadway Spillage']
transport_types = ['Lane/Road Closure Notification', 'Traffic Engineering - Jurisdiction Issue', 'Sign - New', 'Bicycle Issues', 'Traffic Calming', 'Traffic Signal - New/Change', 'Shared Micromobility', 'Sign - New',  'Sign - Traffic Sign Emergency', 'Traffic Engineering - General', 'Concerns in the ROW', 'Obstruction in ROW', 'Construction/Permitting- ROW', 'Tree Issue ROW', 'Sign - Traffic Sign Maintenance', 'Road Markings/Striping - Maintenance', 'Parking Violation Enforcement', 'Construction Items in ROW', 'Parking Machine Issue', 'Speed Management', 'Speed Limit - Changes/Signs', 'View Obstruction at Intersection', 'Sign - School Zone Sign Maintenance', 'Sign - Street Name', 'Sign - Parking Sign Maintenance', 'Road Markings/Striping - New', 'School Zone Flasher - Timing/Maintenance', 'ATD - Booting Complaint', 'Filming Permit - ROW', 'Residential Parking Permit Enforcement', 'School Zone - New/Review/Changes', 'EV PIE Station Issues', 'Valet Permit - ROW', 'Vendor Permit - ROW', 'Parking Permit- ROW', 'ATD - Shared Micromobility', 'Newspaper Rack- ROW', 'Construction items in ROW', 'EV PIP Residential Rebate', 'ATD - Pay-by-Phone App', 'Parking Ticket Complaint', 'School Issues - Crossing Guards', 'Dockless Mobility', 'Construction Items- ROW', 'Guardrail - New', 'Emergency Road Closure Report', 'Mowing Medians', 'Emergency Vehicle Preemption Device']
watershed_types = ['Flooding  Current (Non-Emergency)', 'Flood Report', 'Flooding - Storms During Business Hours', 'Flooding - Storms After Business Hours', 'Channels/Creeks/Drainage Easement', 'Flooding - Past', 'Creek & Pond Vegetation Control', 'Storm Drain Pipe Services', 'Standing Water', 'Ditch/Driveway Pipe Services', 'Flooding Current (Non-Emergency)', 'Drainage - Miscellaneous', 'Town Lake Debris Issues', 'Lost Item in Storm Drainage System']
park_types = ['Park Maintenance - Grounds', 'Park Maintenance - Grounds Plumbing Issues', 'Park Maintenance - Pool Issues', 'Park Maintenance - Cemeteries', 'Mowing City Parks', 'Park Maintenance - Building Issues', 'Park Maintenance - Grounds Electrical Issues', 'Park Maintenance - Building Plumbing Issues', 'Park Maintenance - Building Electrical Issues', 'Park Maintenance - Building A/C & Heating Issues', 'Aquatics Hotline Inquiry']
water_types = ['AW - Water Theft Report', 'AW - Water Waste Report', 'Water Waste Report', 'Water Theft Report']
health_types = ['Public Health - Graffiti Abatement', 'Basic Needs - Appointment', 'Basic Needs - Information Referral',  'Public Health - Environmental Services - City', 'Public Health - Environmental Services - County', 'Graffiti Abatement', 'Public Health - Environmental Services - City', 'Heat Advisory', 'HSEM Heat Advisory']
housing_types = ['LeadSmart Program', 'Neighborhood Home Programs']
fire_types = ['AFD - Wildfire Concern / Presentation']

In [31]:
results_df['Code Check'] = results_df['sr_type_desc'].isin(code_types)
results_df['Animal Check'] = results_df['sr_type_desc'].isin(animal_types)
results_df['Public Types'] = results_df['sr_type_desc'].isin(public_types)
results_df['Resource Check'] = results_df['sr_type_desc'].isin(resource_types)
results_df['Transport Check'] = results_df['sr_type_desc'].isin(transport_types)                                                                
results_df['Watershed Check'] = results_df['sr_type_desc'].isin(watershed_types)
results_df['Park Check'] = results_df['sr_type_desc'].isin(park_types)
results_df['Water Check'] = results_df['sr_type_desc'].isin(water_types)
results_df['Health Check'] = results_df['sr_type_desc'].isin(health_types)
results_df['Housing Check'] = results_df['sr_type_desc'].isin(housing_types)
results_df['Fire Check'] = results_df['sr_type_desc'].isin(fire_types)

In [32]:
results_df.loc[results_df["Code Check"] == True, "Department"] = 'Austin Code'
results_df.loc[results_df["Animal Check"] == True, "Department"] = 'Animal Services'
results_df.loc[results_df["Public Types"] == True, "Department"] = 'Public Works'
results_df.loc[results_df["Resource Check"] == True, "Department"] = 'Austin Resource Recovery'
results_df.loc[results_df["Transport Check"] == True, "Department"] = 'Austin Transportation'
results_df.loc[results_df["Watershed Check"] == True, "Department"] = 'Watershed Check'
results_df.loc[results_df["Park Check"] == True, "Department"] = 'Parks and Recreation'
results_df.loc[results_df["Water Check"] == True, "Department"] = 'Austin Water'
results_df.loc[results_df["Health Check"] == True, "Department"] = 'Public Health'
results_df.loc[results_df["Housing Check"] == True, "Department"] = 'Housing and Planning'
results_df.loc[results_df["Fire Check"] == True, "Department"] = 'Austin Fire'

In [31]:
#top_dept = results_df['Department'].value_counts()
#top_dept

In [32]:
#print(results_df.columns)

In [33]:
results_df = results_df.drop(columns=['Code Check', 'Animal Check', 'Public Types','Resource Check','Transport Check','Watershed Check','Park Check','Water Check','Health Check','Housing Check','Fire Check'])

In [34]:
training_df = results_df.drop(columns=['sr_location', 'Open Time', 'sr_location_lat_long', 'sr_location_council_district', 'sr_closed_date', 'sr_location_x', 'sr_status_desc', 'sr_location_county', 'sr_location_map_page', 'sr_location_long', 'sr_updated_date', 'sr_location_lat', 'sr_status_date', 'sr_location_street_name', 'sr_created_date', 'sr_location_street_number', 'sr_location_city', 'sr_location_y', 'sr_location_map_tile', 'Created Date', 'Closed Date', 'Open Time delta'])

In [35]:
training_df = training_df.drop(columns=['sr_type_desc','sr_number'])

In [36]:
#training_df = training_df.drop(columns=[])

In [36]:
training_df = training_df.rename(columns={'sr_method_received_desc':'Report Method', "sr_location_zip_code":"Zip Code"})

In [37]:
training_df = training_df.rename(columns={'Zip Code':'ZipCode'})

In [39]:
training_df.head(10)

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Total Seconds,Response Time,Department
297886,Phone,78723,2019,December,23,152190.0,Slow Response,Austin Code
297887,Spot311 Interface,78744,2019,December,23,152934.0,Slow Response,Austin Code
297888,Web,78701,2019,December,23,113327.0,Slow Response,Public Works
297889,Spot311 Interface,78701,2019,December,23,121696.0,Slow Response,Austin Resource Recovery
297890,Phone,78701,2019,December,23,155392.0,Slow Response,Austin Code
297891,Phone,78745,2019,December,22,18479378.0,Slow Response,Animal Services
297892,Spot311 Interface,78745,2019,December,22,155819.0,Slow Response,Austin Code
297893,Phone,78746,2019,December,22,1097446.0,Slow Response,Austin Resource Recovery
297894,Spot311 Interface,78745,2019,December,22,117788.0,Slow Response,Animal Services
297895,Spot311 Interface,78660,2019,December,22,162582.0,Slow Response,Animal Services


In [29]:
#read in census data from csv exports from data.census.gov 
mfi_data = "circle/MFI by Zipcode.csv"
pop_data = "circle/Population by Zipcode.csv"

mfi_df = pd.read_csv(mfi_data)
pop_df = pd.read_csv(pop_data)

In [38]:
#drop 0 index row
pop_df = pop_df.drop(0)

In [39]:
mfi_df = mfi_df.drop(0)

In [40]:
#regex to match just zip code 
pop_df['NAME'] = pop_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [41]:
mfi_df['NAME'] = mfi_df['NAME'].str.lstrip('(?<=ZCTA5 )')

In [42]:
pop_df = pop_df.rename(columns={'NAME':'ZipCode','B01003_001E':'TotalPopulation'})

In [43]:
pop_df = pop_df.drop(columns=['GEO_ID','B01003_001M'])

In [44]:
mfi_df = mfi_df.rename(columns={'NAME':'ZipCode','S1903_C03_015E':'MedianFamilyIncome'})

In [45]:
mfi_df = mfi_df.drop(columns=['GEO_ID'])

In [46]:
joined_df = mfi_df.join(pop_df.set_index('ZipCode'), on='ZipCode')

In [47]:
joined_df.head()

,ZipCode,MedianFamilyIncome,TotalPopulation
1,78610,88053,38525
2,78613,115556,81943
3,78617,63405,28930
4,78641,108589,72078
5,78652,103846,5374


In [48]:
training_df = training_df.join(joined_df.set_index('ZipCode'), on='ZipCode')

In [49]:
training_df = training_df.drop(columns = ["Total Seconds"])

In [66]:
training_df.head()

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Response Time,Department,MedianFamilyIncome,TotalPopulation
299101,Phone,78723,2019,December,Night,Slow Response,Austin Code,Median,35725
299102,Spot311 Interface,78744,2019,December,Night,Slow Response,Austin Code,Below Median,50123
299103,Web,78701,2019,December,Night,Slow Response,Public Works,Above Median,10848
299104,Spot311 Interface,78701,2019,December,Night,Slow Response,Austin Resource Recovery,Above Median,10848
299105,Phone,78701,2019,December,Night,Slow Response,Austin Code,Above Median,10848


In [63]:
#training_df.loc[training_df['MedianFamilyIncome'] == '-']

,Report Method,ZipCode,Request Year,Request Month,Time of Day,Response Time,Department,MedianFamilyIncome,TotalPopulation


In [62]:
training_df.drop(training_df[training_df['MedianFamilyIncome'] == '-'].index, inplace = True)

In [67]:
#training_df['MedianFamilyIncome'].value_counts()

Above Median    176502
Median           80525
Below Median     64042
0                 4781
Name: MedianFamilyIncome, dtype: int64

In [52]:
#training_df.dtypes

Report Method         object
ZipCode               object
Request Year           int64
Request Month         object
Time of Day           object
Response Time         object
Department            object
MedianFamilyIncome    object
TotalPopulation       object
dtype: object

In [64]:
training_df['MedianFamilyIncome'] = pd.to_numeric(training_df['MedianFamilyIncome'])

In [65]:
eighty_mfi = 57260
onetwenty_mfi = 85891
conditions = [
    (training_df['MedianFamilyIncome'] < eighty_mfi),
    (training_df['MedianFamilyIncome'] >= eighty_mfi) & (training_df['MedianFamilyIncome'] <= onetwenty_mfi),
    (training_df['MedianFamilyIncome'] > onetwenty_mfi)
    ]
values = ['Below Median', 'Median', 'Above Median']
training_df['MedianFamilyIncome'] = np.select(conditions, values)

In [68]:
#training_df.rename(columns={'index':'Index'}, inplace =True)
#command = 'Index serial primary key, Report Method text, Zip Code varchar(5), Request Year int, Total Seconds interval, Response Time text, Department text'


training_df.to_sql('database', con=engine, if_exists='replace')

engine.execute("SELECT * FROM database").fetchall()

[(299101, 'Phone', '78723', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Median', '35725'),
 (299102, 'Spot311 Interface', '78744', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Below Median', '50123'),
 (299103, 'Web', '78701', 2019, 'December', 'Night', 'Slow Response', 'Public Works', 'Above Median', '10848'),
 (299104, 'Spot311 Interface', '78701', 2019, 'December', 'Night', 'Slow Response', 'Austin Resource Recovery', 'Above Median', '10848'),
 (299105, 'Phone', '78701', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Above Median', '10848'),
 (299106, 'Phone', '78745', 2019, 'December', 'Night', 'Slow Response', 'Animal Services', 'Median', '61907'),
 (299107, 'Spot311 Interface', '78745', 2019, 'December', 'Night', 'Slow Response', 'Austin Code', 'Median', '61907'),
 (299108, 'Phone', '78746', 2019, 'December', 'Night', 'Slow Response', 'Austin Resource Recovery', 'Above Median', '28608'),
 (299109, 'Spot311 Interface', '78745', 2019, 'De

Machine Learning Model

In [65]:
# Create our features
X = training_df.drop(columns = ["Response Time"])

# Create dummy variables
X = pd.get_dummies(X)

# Create our target
y = training_df[["Response Time"]]

In [66]:
# Check the balance of our target values
y["Response Time"].value_counts()

Quick Response    163161
Slow Response     163158
Name: Response Time, dtype: int64

In [67]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [68]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rand_for_class = BalancedRandomForestClassifier(n_estimators = 100, random_state = 1)
rand_for_class.fit(X_train, y_train.values.ravel())

BalancedRandomForestClassifier(random_state=1)

In [69]:
# Calculated the balanced accuracy score
y_prediction = rand_for_class.predict(X_test)
balanced_accuracy_score(y_test, y_prediction)

0.7458487775807341

In [70]:
# Display the confusion matrix
confusion_matrix(y_test, y_prediction)

array([[29475, 11090],
       [ 9635, 31380]], dtype=int64)

In [71]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_prediction))

                      pre       rec       spe        f1       geo       iba       sup

Quick Response       0.75      0.73      0.77      0.74      0.75      0.55     40565
 Slow Response       0.74      0.77      0.73      0.75      0.75      0.56     41015

   avg / total       0.75      0.75      0.75      0.75      0.75      0.56     81580



In [72]:
# List the features sorted in descending order by feature importance
features = rand_for_class.feature_importances_
sort_features = np.argsort(features)[::-1]

for i in sort_features:
    print("{}: ({})".format(X.columns.values[i], features[i]))

Time of Day: (0.30927842544775386)
Department_Austin Code: (0.22067609122050197)
Department_Austin Resource Recovery: (0.06632926343874218)
Request Year: (0.03666316915195963)
Department_Austin Transportation: (0.03330677435601429)
Department_Animal Services: (0.02709081169512377)
Department_Public Works: (0.01810279288464265)
Report Method_Phone: (0.01413506625816749)
Department_Watershed Check: (0.013675486677654699)
Report Method_Spot311 Interface: (0.013119747185082265)
Request Month_June: (0.0110908375024238)
Request Month_May: (0.010533834332874198)
Request Month_July: (0.009422343843603673)
Department_Parks and Recreation: (0.009366612148446899)
Request Month_October: (0.009136299983414392)
Department_Public Health: (0.009073703270490901)
Request Month_April: (0.008905010915330748)
Request Month_August: (0.008677027454759942)
Request Month_September: (0.008332351049512653)
Report Method_Web: (0.008284581503921041)
Request Month_December: (0.008263337503910234)
Request Month_Nove